IA & Data science (LU3IN0226) -- 2024-2025
--------
*&copy; Equipe pédagogique: Christophe Marsala, Olivier Schwander, Jean-Noël Vittaut.*


# TD-TME 6 : Apprentissage pour le texte

<font size="+1" color="RED"><b>[Q]</b></font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

TOUTAIN Yanis

<font color="RED" size="+1"><b>[Q]</b></font> **Renommer ce notebook**

Tout en haut de cette page, cliquer sur <tt>tme-06</tt> et rajouter à la suite de <tt>tme-06</tt> les noms des membres du binômes séparés par un tiret.

<font color="RED" size="+1">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *tme-06-Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: il faut rendre le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers vous les rendez séparément.

**Echancier pour la soumission de votre compte-rendu:**
- le compte-rendu d'une séance doit être remis obligatoirement <font color="RED">avant la séance suivante</font>.

**Le compte-rendu est soumis sur la page Moodle.**

In [5]:
# - - - - - - - - - - - - - - - - - -
# imports utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mtpl
%matplotlib inline  

import math
import time
import sys

# Les instructions suivantes sont utiles pour recharger automatiquement 
# le code modifié dans les librairies externes
%load_ext autoreload
%autoreload 2

# - - - - - - - - - - - - - - - - - -
# Information sur l'environnent utilisé ici:
print("Version python et des librairies:")
print("\tPython ",sys.version)
print("\tpandas: ",pd.__version__)
print("\tnumpy: ",np.__version__)
print("\tmatplotlib: ",mtpl.__version__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Version python et des librairies:
	Python  3.11.2 (main, Nov 30 2024, 21:22:50) [GCC 12.2.0]
	pandas:  2.2.0
	numpy:  1.24.2
	matplotlib:  3.6.3


In [6]:
# Importation de votre librairie iads:
# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')   # iads doit être dans le répertoire père du répertoire courant !

# Importation de la librairie iads
import iads as iads

# importation de Classifiers
from iads import Classifiers as classif

# importation de utils
from iads import utils as ut

# importation de evaluation
from iads import evaluation as ev



# Objectifs de ce TME

<div class="alert alert-block alert-warning">
Ce TME a pour but d'implémenter des fonctions pour traiter un corpus textuel et un nouvel algorithme d'apprentissage vu dans le cours 6. 

Pour expérimenter, on utilise la base `SMS spam Collection` qui contient 5572 messages associés à 2 labels: `spam` et `ham`. 
</div>

In [7]:
# Chargement du dataset

df_spam = pd.read_csv('data/spam.csv', sep='\t', encoding = 'latin1')
df_spam

,label,message
0,ham,Go until jurong point crazy.. Available only ...
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,Nah I don't think he goes to usf he lives aro...
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,Pity * was in mood for that. So...any other s...
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
# Valeurs du label:
df_spam['label'].unique()

array(['ham', 'spam'], dtype=object)

In [9]:
# On met les labels dans une liste car cela sera utile:
les_labels = df_spam['label'].unique()

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant `value_counts` (voir la doc de la librairie pandas), afficher le nombre d'exemples de chaque classe dans la base.

In [10]:
df_spam['label'].value_counts()

label
ham     4825
spam     747
Name: count, dtype: int64

## Prétraitements

### Nettoyage des données

Pour pouvoir travailler sur les messages, on commence par les nettoyer : il faut enlever les caractères de ponctuations, les articles, les mots trop courants, etc. (voir le cours 6).

Pour traiter les données textuelles (colonne `message` du dataset), on utilise les fonctions de la librairie `string`.

In [11]:
import string

print("Caractères de ponctuations : ", string.punctuation)

print("Mise en minuscules (pour homogénéiser l'écriture : ", "May the Force be with you!".lower())

print("Découper une phrase avec espace (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split())

print("Découper une phrase avec apostrophe (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split("'"))



Caractères de ponctuations :  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Mise en minuscules (pour homogénéiser l'écriture :  may the force be with you!
Découper une phrase avec espace (retourne une liste): ['LU3IN026', 'est', "l'UE", "d'IA", 'et', 'Sciences', 'des', 'données.']
Découper une phrase avec apostrophe (retourne une liste): ['LU3IN026 est l', 'UE d', 'IA et Sciences des données.']


In [12]:
# Avec le dataframe du dataset:
print("Premier message du train: ",df_spam['message'].iloc[0])

print("Résultat d'un découpage: ",df_spam['message'].iloc[0].split())

Premier message du train:  Go until jurong point  crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Résultat d'un découpage:  ['Go', 'until', 'jurong', 'point', 'crazy..', 'Available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet...', 'Cine', 'there', 'got', 'amore', 'wat...']


In [13]:
# Certains éléments d'une phrase ne sont pas utiles pour le traitement, par exemple en anglais:
mots_inutiles = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `nettoyage` qui prend une chaîne de caractères et rend la chaîne après nettoyage: 1) mise en minuscules 2) remplacement des caractères de ponctuation par un espace (SAUF l'apostrophe qui doit rester car elle va être utilisée pour les mots inutiles).


In [14]:
def nettoyage(chaine):

    res = chaine.lower()
    for car in string.punctuation:
        if(car != "'" and car in chaine):
            res=res.replace(car,' ')
    return res

In [15]:
nettoyage("LU3IN026 est l'UE d'IA et Sciences des données.")

"lu3in026 est l'ue d'ia et sciences des données "

In [16]:
nettoyage(df_spam['message'].iloc[0])

'go until jurong point  crazy   available only in bugis n great world la e buffet    cine there got amore wat   '

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `text2vect` qui prend une chaîne de caractères ainsi qu'une liste de mots inutiles et rend la liste composée par les mots de cette chaîne obtenus, après son nettoyage et après avoir enlevé les mots inutiles.



In [17]:
def text2vect(chaine,mots_inutiles):
    chaine_nettoye = nettoyage(chaine)
    res = chaine_nettoye.split()
    return [mot for mot in res if mot not in mots_inutiles]

In [18]:
text2vect("May the Force be with you!",mots_inutiles)

['may', 'force']

In [19]:
text2vect("You shan't pass!",mots_inutiles)

['pass']

In [20]:
text2vect(df_spam['message'].iloc[0],mots_inutiles)

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'n',
 'great',
 'world',
 'la',
 'e',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

<font color="RED" size="+1"><b>[Q]</b></font> Ajouter une nouvelle colonne de nom `les_mots` au dataframe `df_spam` pour laquelle chaque ligne contient le résultat de l'application de `text2vect` sur le message de l'exemple correspondant.


In [21]:
# A COMPLETER 
df_spam["les_mots"] = df_spam["message"].apply(text2vect, args=(mots_inutiles,))
# ----------------------------------------------------------

df_spam

,label,message,les_mots
0,ham,Go until jurong point crazy.. Available only ...,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,Nah I don't think he goes to usf he lives aro...,"[nah, think, goes, usf, lives, around, though]"
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[ì, b, going, esplanade, fr, home]"
5569,ham,Pity * was in mood for that. So...any other s...,"[pity, mood, suggestions]"
5570,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, i'd, interested, ..."


### Découpage du dataset en 2 ensembles train et test

<font color="RED" size="+1"><b>[Q]</b></font> Pour mettre au point nos fonctions dans ce TME, on va travailler sur une partie du dataset, par exemple, on peut prendre 5% des exemples (en respectant la distribution des classes, donc en prenant 5% d'exemples de chaque label).
Compléter le code ci-dessous pour définir les 2 variables `df_train` et `df_test` qui contiendront chacune 1 dataframe, partie de `df_spam`. Pour construire `df_train` on prend aléatoirement 5% des exemples de chaque classe, comme on l'a déjà fait dans un TME précédent (cf. `np.random.shuffle`).


In [22]:
np.random.seed(42)

# pourcentage d'exemples de chaque classe à garder:
taux = 0.05    # ici on prend 5% 

# déclaration des variables qui seront initialisées dans la boucle:
df_train = None  
df_test = None
for l in les_labels:
    nb_total = df_spam['label'].value_counts()[l]
    nb_pris = int(nb_total*taux) 
    print(f"Nombre d'exemples du label {l} pris pour apprendre: {nb_pris}")

    les_ids = df_spam[df_spam['label']==l].index.to_list()
    np.random.shuffle(les_ids)

    selected_ids = les_ids[:nb_pris]
    cpt = df_spam.iloc[selected_ids]
    df_train = pd.concat([df_train,cpt])

df_test = df_spam[~df_spam.index.isin(df_train.index)]
    # ###########################################################
    
# Résultat:
print(f"Dimension de df_train:\t{df_train.shape}")
print(f"Dimension de df_test:\t{df_test.shape}")

Nombre d'exemples du label ham pris pour apprendre: 241
Nombre d'exemples du label spam pris pour apprendre: 37
Dimension de df_train:	(278, 3)
Dimension de df_test:	(5294, 3)


In [23]:
df_train

,label,message,les_mots
3714,ham,I am late so call you tomorrow morning.take ca...,"[late, call, tomorrow, morning, take, care, sw..."
1311,ham,U r too much close to my heart. If u go away i...,"[u, r, much, close, heart, u, go, away, shatte..."
548,ham,Wait &lt;#&gt; min..,"[wait, lt, gt, min]"
1324,ham,Can you call me plz. Your number shows out of ...,"[call, plz, number, shows, coveragd, area, urg..."
3184,ham,MAYBE IF YOU WOKE UP BEFORE FUCKING 3 THIS WOU...,"[maybe, woke, fucking, 3, problem]"
...,...,...,...
1852,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, 750, poun..."
2582,spam,3 FREE TAROT TEXTS! Find out about your love l...,"[3, free, tarot, texts, find, love, life, try,..."
3418,spam,Do you want a new Video phone? 600 anytime any...,"[want, new, video, phone, 600, anytime, networ..."
3422,spam,Had your mobile 10 mths? Update to latest Oran...,"[mobile, 10, mths, update, latest, orange, cam..."


## Calculs de probabilités

À partir d'ici on va travailler sur les données d'apprentissage (`df_train`) pour déterminer les probabilités qui serviront à classer les messages.

<font color="RED" size="+1"><b>[Q]</b></font> Construire la liste de tous les mots présents dans tous les vecteurs de `df_train`, chaque mots doit n'apparaître qu'une seule fois dans la liste obtenue. Cette liste sera stockée une fois triée dans la variable `index_mots` (et on l'appelle par la suite "index de mots").


In [24]:
index_mots = np.unique([mot for sous_liste in df_train["les_mots"] for mot in sous_liste])

In [25]:

print("Nombre de mots trouvés: ", len(index_mots))
print("Les 10 premiers :", index_mots[0:10]) 

# pour contrôler:
for i in range(30,len(index_mots),100):
    print("\ten position ",i," --> ", index_mots[i])


Nombre de mots trouvés:  1377
Les 10 premiers : ["'" "'t" '0' '00' '01223585334' '0207' '03' '07801543489' '0800'
 '08000930705']
	en position  30  -->  10p
	en position  130  -->  al
	en position  230  -->  brain
	en position  330  -->  considering
	en position  430  -->  emerging
	en position  530  -->  go
	en position  630  -->  interesting
	en position  730  -->  lucky
	en position  830  -->  next
	en position  930  -->  porn
	en position  1030  -->  screaming
	en position  1130  -->  sub
	en position  1230  -->  tv
	en position  1330  -->  workin


Chaque message va maintenant être représenté comme un vecteur de valeurs 0 ou 1: ce vecteur possède autant de colonnes qu'il y a de mots dans `index_mots`. Ce vecteur est construit ainsi: pour un exemple $i$, la valeur de la colonne $j$ du vecteur vaudra 1 si la liste de mots de l'exemple $i$ contient le mot en position $j$ dans `index_mots`, ou 0 dans le cas contraire.

*Remarque:* même si un mot apparaît plusieurs fois dans la liste, cela ne compte que pour 1.

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `df2array` qui prend un dataframe `df` contenant la colonne `les_mots` ainsi qu'un index de mots et rend le `np.array` correspondant aux vecteurs de valeurs représentant les exemples de `df`. Les mots de `les_mots` qui ne sont pas dans l'index de mots ne sont pas pris en compte.  


In [26]:
def df2array(df,index_mots):
    mot_to_index = {mot: i for i, mot in enumerate(index_mots)}
    res = []

    for mots in df["les_mots"]:
        v = np.zeros(len(index_mots), dtype = int)
        for mot in mots:
            if mot in mot_to_index:
                v[mot_to_index[mot]] = 1

        res.append(v)

    return np.array(res)

In [27]:
mat_train = df2array(df_train,index_mots)

In [28]:
mat_train.shape

(278, 1377)

In [29]:
print("Message 0:", df_train['les_mots'].iloc[0])

print("\nPositions non nulles dans le vecteur d'index:")

for i in range(0, len(mat_train[0])):
    if mat_train[0][i] == 1:
        print("\tcolonne ",i," : ", index_mots[i])
    

Message 0: ['late', 'call', 'tomorrow', 'morning', 'take', 'care', 'sweet', 'dreams', 'u', 'ummifying', 'bye']

Positions non nulles dans le vecteur d'index:
	colonne  245  :  bye
	colonne  248  :  call
	colonne  265  :  care
	colonne  412  :  dreams
	colonne  679  :  late
	colonne  791  :  morning
	colonne  1150  :  sweet
	colonne  1154  :  take
	colonne  1209  :  tomorrow
	colonne  1238  :  u
	colonne  1244  :  ummifying


In [30]:
# on se rappelle qu'il est possible d'extraire les vecteurs correspondant à un label donné, par exemple:
len(mat_train[df_train['label']=='ham'])

241

<font color="RED" size="+1"><b>[Q]</b></font> Construire un dictionnaire qui donne, pour chaque label $l$, pour chaque mot $m$ de l'index des mots, la fréquence de $m$ parmi les exemples de `df_train` qui ont le label $l$.


*Remarque*: penser à une solution dans boucle for sur les exemples...


In [31]:
frequences = dict()
for l in les_labels:
    frequences[l] = np.array([0.0] * len(index_mots))

    count = mat_train[df_train['label'] == l].sum(axis=0)
    size_count = len(mat_train[df_train['label'] == l])
    for id_element in range(len(count)):
        frequences[l][id_element] = count[id_element] / size_count

In [32]:
# Affichage de quelques valeurs de fréquence non nulles 
print("Seuls les 10 premiers non nuls sont affichés.")        
for l in frequences:
    nb= 0
    print("Pour le label",l, ":")
    for j in range(0,len(frequences[l])):
        if frequences[l][j] != 0:
            if (nb < 10):
                print(f'\t {index_mots[j]}:\t {frequences[l][j]:0.6f}')
            nb +=1


Seuls les 10 premiers non nuls sont affichés.
Pour le label ham :
	 ':	 0.004149
	 't:	 0.004149
	 1:	 0.016598
	 100:	 0.004149
	 1st:	 0.004149
	 2:	 0.058091
	 2marrow:	 0.004149
	 2nd:	 0.004149
	 3:	 0.020747
	 30:	 0.004149
Pour le label spam :
	 0:	 0.027027
	 00:	 0.054054
	 01223585334:	 0.027027
	 0207:	 0.027027
	 03:	 0.027027
	 07801543489:	 0.027027
	 0800:	 0.027027
	 08000930705:	 0.054054
	 08000938767:	 0.027027
	 08002888812:	 0.027027


In [33]:
print("Frequences max:")
print("\tpour ham:", max(frequences['ham']), "pour le mot",index_mots[np.argmax(frequences['ham'])])
print("\tpour spam:", max(frequences['spam']), "pour le mot",index_mots[np.argmax(frequences['spam'])])


Frequences max:
	pour ham: 0.18672199170124482 pour le mot u
	pour spam: 0.4864864864864865 pour le mot call


## Classification de textes

<div class="alert alert-block alert-warning">
On considère deux variables $X$ et $Y$ :
    <ul>
        <li>$X$ est un message</li>
        <li>$Y$ est le label d'un message et peut prendre 2 valeurs : <code>'ham'</code> et <code>'spam'</code></li>
    </ul>

Avec les fonctions précédentes on peut représenter les messages d'un corpus de documents sous la forme d'un vecteur $X \in \{0, 1\}^p$ où $p$ est le nombre total de mots de l'index. Le $i$ème message est représenté par le vecteur ${\bf x}_i = [x_{i1} \dots x_{ip}]$, où $x_{ij}$ vaut 1 si le $j$ème mot de l'index est présent dans le message $i$, et 0 sinon.
    
Comme vu en cours, pour un classifieur bayésien, nous devons estimer, à partir de la base <code>df_train</code>, les probabilités $p(ham)$, $p(spam)$, $p(X |ham)$ et $p(X | spam)$.

Les 2 premières sont simples à calculer : on compte le nombre de fois où le label apparaît parmi les exemples.

Pour un label $Y$ et un exemple ${\bf x}$, le calcul de $p({\bf x} | Y)$ se fait en utilisant les probabilités $p(mot | Y)$ de tous les mots qui composent ${\bf x}$ (et qui sont des mots de l'index des mots). $p(mot | Y)$ est la probabilité que le mot <code>mot</code> apparaisse dans un message sachant que ce message appartient à la classe $Y$.
    
On pose ainsi

$$ p({\bf x} | Y) = \prod_{mot \in index\_mots} p(mot | Y)^{x_{mot}} \left(1 - p(mot | Y)\right)^{1 - x_{mot}} $$

où $x_{mot}$ correspond à $1$ ou $0$ selon que le `mot` apparaît dans ${\bf x}$ ou pas. Ce terme permet de retenir soit la probabilité $p(mot | Y)$ si le mot est dans ${\bf x}$, soit la probabilité qu'il n'y soit pas ($1-p(mot |Y)$).
    
Une fois que $p({\bf x} | Y)$ est calculée, on peut estimer $p(Y|X)$ grâce au théorème de Bayes (cf. cours 6):

$$p(Y|{\bf x}) = p(Y) p({\bf x} | Y)$$

avec $Y$ qui vaut soit 'ham', soit 'spam'.    

    
Une fois $p(Y{{\bf x}})$ calculée pour chaque valeur de label, pour prédire le label de ${\bf x}$ on choisit le label qui possède la plus forte probabilité.
</div>

In [34]:
for l in ['ham', 'spam']:
    # #################### A COMPLETER 
    proba = len(df_train[df_train["label"] == l])/len(df_train)
    ##########################
    print(f'p({l:4}) = {proba:0.4f}')


p(ham ) = 0.8669
p(spam) = 0.1331


<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_mot` qui étant donné un mot, un label, une liste de mots, et un dictionnaire avec les fréquences des mots par label (comme `frequences` et compatible avec la liste de mots) rend $p(mot| label)$ la probabilité du mot d'appartenir au label donné.


In [35]:
def proba_mot(mot,label,index_mots,frequences):
    if mot in index_mots and label in frequences:
        index = np.where(index_mots == mot)[0][0]
        return frequences[label][index]
    else:
        return 0

In [36]:
# probabilité d'un mot qui n'est pas dans l'index:
proba_mot("toto", "ham", index_mots, frequences)

0

In [37]:
# probabilité d'un mot de l'index pour un label qui n'existe pas:
proba_mot("visit", "cookie", index_mots, frequences)

0

In [38]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("call", "ham", index_mots, frequences)

0.05394190871369295

In [39]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("call", "spam", index_mots, frequences)

0.4864864864864865

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_exemple` qui étant donné un exemple représenté sous la forme d'une liste de mots, un label, une liste de mots, et un dictionnaire comme `frequences` (compatible avec la liste de mots) rend $p(exemple|label)$ la probabilité de l'exemple d'appartenir au label.


In [40]:
def proba_exemple(exemple,label,index_mots,frequences):
    res = 1
    for mot in index_mots:
        if mot in exemple:
            res=res*proba_mot(mot,label,index_mots,frequences)
        else:
            res=res*(1-proba_mot(mot,label,index_mots,frequences))
    return res

In [41]:
proba_exemple(df_train['les_mots'].iloc[10], 'ham', index_mots,frequences)

1.743474723482465e-16

In [42]:
for i in range(0,len(df_train)):
    p_spam = proba_exemple(df_train['les_mots'].iloc[i], 'spam', index_mots,frequences)
    p_ham = proba_exemple(df_train['les_mots'].iloc[i], 'ham', index_mots,frequences)
    if (p_spam>0) and (p_ham>0):
        print("Exemple",i,": p(ham)=",p_ham,"\tp(spam)=",p_spam)


Exemple 155 : p(ham)= 0.00034444356666484764 	p(spam)= 5.691806029912198e-09
Exemple 230 : p(ham)= 6.211338526794225e-10 	p(spam)= 2.3229490990356926e-12


<font color="RED" size="+1"><b>[Q]</b></font> À partir de `mat_train` et de `frequences` et en utilisant des opérations matricielles, on peut aussi calculer les probabilités pour chaque label de tous les exemples. En terme de temps de calcul, cela peut être plus efficace.

Donner l'instruction permettant de calculer toutes ces probabilités, puis vérifier que vous trouver les mêmes valeurs pour les exemples précédents.


In [74]:
toutes_probas = dict()
for l in les_labels:
    
    frequences_label = frequences[l]
    
    # Calculer les scores en effectuant un produit scalaire avec mat_train
    probas_label = np.dot(mat_train, frequences_label)
    
    exp_scores = np.exp(probas_label)
    
    probas_label_normalized = exp_scores / np.sum(exp_scores, axis=0, keepdims=True)
    
    # Sauvegarder les probabilités dans le dictionnaire
    toutes_probas[l] = probas_label_normalized


In [75]:
for i in range(0,len(df_train)):
    p_spam = toutes_probas['spam'][i]
    p_ham = toutes_probas['ham'][i]
    if (p_spam>0) and (p_ham>0):
        print("Exemple",i,": p(ham)=",p_ham,"\tp(spam)=",p_spam)



Exemple 0 : p(ham)= 0.004446708967079718 	p(spam)= 0.0042965193774574485
Exemple 1 : p(ham)= 0.00437351377811864 	p(spam)= 0.002641425573375015
Exemple 2 : p(ham)= 0.0034523398944196187 	p(spam)= 0.002246012898483751
Exemple 3 : p(ham)= 0.0035540858117855344 	p(spam)= 0.003753430090404869
Exemple 4 : p(ham)= 0.003244014375045899 	p(spam)= 0.002307543701696923
Exemple 5 : p(ham)= 0.0034238083228240405 	p(spam)= 0.002570991788290878
Exemple 6 : p(ham)= 0.004994547598983924 	p(spam)= 0.003191554015975859
Exemple 7 : p(ham)= 0.0034955836735742333 	p(spam)= 0.002307543701696923
Exemple 8 : p(ham)= 0.0030992756223870268 	p(spam)= 0.002246012898483751
Exemple 9 : p(ham)= 0.003568863694121789 	p(spam)= 0.002641425573375015
Exemple 10 : p(ham)= 0.003367450621204499 	p(spam)= 0.002307543701696923
Exemple 11 : p(ham)= 0.003244014375045899 	p(spam)= 0.002246012898483751
Exemple 12 : p(ham)= 0.004932760174188331 	p(spam)= 0.003191554015975859
Exemple 13 : p(ham)= 0.003909985084235422 	p(spam)= 0.00

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant les fonctions écrites, calculer le taux de bonne prédiction du classifieur bayésien naïf pour chaque valeur de label pour le dataset d'apprentissage. 

*Remarque*: une solution efficace peut ne pas utiliser toutes les fonctions précédentes...

In [80]:
# La variable suivante permettra de stocker les résultats
taux = dict()
toutes_probas = dict()
for l in les_labels:
    taux[l] = dict()
    taux[l][True] = 0    # nombre de bien classés
    taux[l][False] = 0   # nombre de mal classés
    toutes_probas[l] = np.zeros(1)
tic = time.time()

for index, row in df_train.iterrows():
    exemple = row["les_mots"]
    vrai_label = row["label"]

    probabilites = {}
    for label in les_labels:
        probabilites[label] = proba_exemple(exemple, label, index_mots, frequences)

    # Trouver le label avec la probabilité la plus élevée
    predicted_label = max(probabilites, key=probabilites.get)

    toutes_probas[predicted_label] = np.append(toutes_probas[predicted_label], probabilites[predicted_label])

    for l in les_labels:
        if l == vrai_label:
            if l == predicted_label:
                taux[l][True] += 1  # bien classé
            else:
                taux[l][False] += 1  # mal classé

    for l in les_labels:
        total = taux[l][True] + taux[l][False]
        taux[l]['Taux'] = taux[l][True] / total if total != 0 else 0
# #############################################
toc = time.time()
print("Résultats des classements : ", taux)

print("Temps: ",toc-tic)
# le résultat total peut prendre un certain temps...


Résultats des classements :  {'ham': {True: 241, False: 0, 'Taux': 1.0}, 'spam': {True: 37, False: 0, 'Taux': 1.0}}
Temps:  3.3476459980010986


<font color="RED" size="+1"><b>[Q]</b></font> Même question pour calculer le taux de bonne classification pour le dataset de test.

*Remarque*: L'index des mots ainsi que la matrice des fréquences restent les mêmes, ce sont toujours celles construites à partir de la base d'apprentissage. Par contre, la matrice des présences doit, elle, être recalculées.

In [81]:
# La variable suivante permettra de stocker les résultats
taux = dict()
toutes_probas = dict()
for l in les_labels:
    taux[l] = dict()
    taux[l][True] = 0    # nombre de bien classés
    taux[l][False] = 0   # nombre de mal classés
    toutes_probas[l] = np.zeros(1)
tic = time.time()

mat_train = df2array(df_test, index_mots)

for index, row in df_test.iterrows():
    exemple = row["les_mots"]
    vrai_label = row["label"]

    probabilites = {}
    for label in les_labels:
        probabilites[label] = proba_exemple(exemple, label, index_mots, frequences)

    # Trouver le label avec la probabilité la plus élevée
    predicted_label = max(probabilites, key=probabilites.get)

    toutes_probas[predicted_label] = np.append(toutes_probas[predicted_label], probabilites[predicted_label])

    for l in les_labels:
        if l == vrai_label:
            if l == predicted_label:
                taux[l][True] += 1  # bien classé
            else:
                taux[l][False] += 1  # mal classé

    for l in les_labels:
        total = taux[l][True] + taux[l][False]
        taux[l]['Taux'] = taux[l][True] / total if total != 0 else 0


# #############################################
toc = time.time()
print("Résultats des classements : ", taux)

print("Temps: ",toc-tic)
# le résultat total peut prendre un certain temps...


# Il reste à calculer les taux de bonne classification par label

Résultats des classements :  {'ham': {True: 4552, False: 32, 'Taux': 0.9930191972076788}, 'spam': {True: 195, False: 515, 'Taux': 0.2746478873239437}}
Temps:  63.77068781852722


## Evaluation du classifieur Naive Bayes

Pour tout ce que l'on a fait jusque-là, on a travaillé sur tout le dataset, pour pouvoir l'évaluer il est nécessaire de le séparer en ensemble d'apprentissage et ensemble de test.

<font color="RED" size="+1"><b>[Q]</b></font> Découper aléatoirement `df_spam` en 2 parties égales, chacune contenant des exemples des 2 labels, avec la même distribution des labels dans chaque partie. 
Une des parties sert à apprendre l'index des mots et leurs fréquences.
L'autre partie n'est utilisée que pour calculer le taux de bonne classification par label.

Donner ensuite les taux de bonne classification par label pour l'ensemble de train et pour l'ensemble de test.

*Remarque*: certains mots de la partie de test pourront ne pas être présents dans l'index de mots car ils peuvent être absents de la partie d'apprentissage.

<font color="RED" size="+1"><b>[Q]</b></font> Mettre en place une approche par validation croisée pour évaluer le taux de bonne classification moyen de cette approche. 

<font color="RED" size="+1"><b>[Q]</b></font> Comparer les résultats obtenus (taux de bonne classification avec la validation croisée, temps de calcul) avec ceux obtenus avec l'application d'un classifieur par $k$ plus proches voisins. Pour cela, la `mat_spam` doit être utilisée comme description des données. 